In [1]:
!pip install scrapy
!pip install snscrape
#cd /data/notebook_files/skysport/

You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/python/envs/default/bin/python -m pip install --upgrade pip' command.


In [2]:
!scrapy startproject skysport

Error: scrapy.cfg already exists in /data/notebook_files/skysport


In [10]:
cd /data/notebook_files/skysport

/data/notebook_files/skysport


In [ ]:
!rm scrapedUrls.json
!scrapy crawl titles -o scrapedUrls.json\

In [ ]:
import pandas as pd
import requests
import json

dataframe = pd.read_json(path_or_buf='./scrapedUrlsX.json')

#print(dataframe['tags'].head(10))


token = '9592c1c916514c54a712c48fe03a14b4'

uri = 'https://api.football-data.org/v4/competitions/SA/matches?season=2020'
headers = { 'X-Auth-Token': token}

matches = requests.get(uri, headers=headers)
#for match in matches.json()['matches']:
  #print(match)
    

uri = 'https://api.football-data.org/v4/competitions/SA/teams?season=2020'
headers = { 'X-Auth-Token': token}

teams = requests.get(uri, headers=headers)

for team in teams.json()['teams']:
  print(team)
    

teams = [team['shortName'] for team in teams.json()['teams']]

teams_dict = { i : pd.DataFrame() for i in teams }


def adjust_name(name):

    return name.split(' ')[0].lower()

for team in teams:
    
    teams_dict[team] = dataframe[[(adjust_name(team) in x) for x in dataframe.tags]]

In [ ]:
!pip install selenium
from selenium import webdriver

In [ ]:
!pip install pyvirtualdisplay
!sudo apt-get update
!sudo apt -y --fix-broken install
!sudo apt-get -y install xvfb

In [99]:
import os
os.chmod("/data/notebook_files/chromedriver", 0o777)

In [ ]:
!sudo apt -y install -f
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!sudo dpkg -i google-chrome-stable_current_amd64.deb

In [1]:
def remove_prefix(string):

    return string.replace("AC ", "").replace("Calcio ", '').replace("FC ", "")


def remove_adds(string):

    if '+' in string:

        return string.split("+")[0]

    elif '-' in string:

        return string.split("-")[0]

    else:

        return string

    
def get_numbers(string): 

    listx =  ' '.join([remove_adds(n) for n in string.split(' ')])
    sentence = listx
    #print('LIST')
    #print(listx,'\n')
    numbers = [float(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    #print('NUMBERS')
    #print(numbers)
    numbers = [(int(n) if n.is_integer() else n) for n in numbers]
    #print(numbers)
    try:
        if(len(numbers) == 9) or (len(numbers) == 11):
            numbers[2] /= 1000
        else:
            numbers[3] /= 1000
    except IndexError:
        pass
    numbers = [n for n in numbers if n<1000]

    return numbers


def get_team_player(string, leaderboard):
    name = ""
    for team in leaderboard.Team:
        if team in string:
            string = string.replace(team,'')
            name = team
            break

    return (string,name)

def get_table(i,l,leaderboard,driver):
    try:
        pagination = driver.find_elements('xpath',"//ul//li[contains(@class,'page')]")
        #print("PAGINAA")
        #print(pagination[i].text)
        pagination[i].click()
        #pagination = pagination.split("\n")[-1]
        #print(pagination)


    except NoSuchElementException:
        return True

    except ElementClickInterceptedException:
        pass


    j = 1


    while True:
        try:
            b = driver.find_element('xpath',f"//div[@id='league-players']/table/tbody/tr[{j}]").text


            
            (c, team) = get_team_player(b,leaderboard)
            players = [str for str in c.split(' ') if (str.isalpha())]
            
            c = get_numbers(b)
            c.insert(1, ' '.join(players))
            c.insert(2, team)
        
            l.append(c) 
            #print(c)
            j+=1
            
            #time.sleep(2)
        except NoSuchElementException:
            break

In [5]:
!pip install webdriver-manager

In [10]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#rom pyvirtualdisplay import Display
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
#from sqlalchemy import except_
import time
import re
import pandas as pd

#display = Display(size=(1920, 1080))  

#display.start()
url = 'https://understat.com/league/Serie_A/2020'
options = webdriver.ChromeOptions()
#options.add_argument('--disable-dev-shm-usage')      
#options.binary_location = "/usr/bin/google-chrome-stable"
#chrome_driver_binary = "/data/notebook_files/chromedriver"
#chrome = ChromeDriverManager().install()
d = webdriver.Chrome(chrome,chrome_options=options)
d.get(url)

def get_tables(s_date, e_date,driver):

    driver = webdriver.Chrome(chrome,chrome_options=options)
    driver.get(url)

    driver.find_element('name','date-start').click()


    sDate  = driver.find_element('name','date-start') # set start date input element into variable
    sDate.clear() #clear existing entry
    sDate.send_keys(s_date) #add custom entry


    driver.find_element('name','date-end').click()

    eDate  = driver.find_element('name','date-end') # set start date input element into variable
    eDate.clear() #clear existing entry
    eDate.send_keys(e_date) #add custom entry

    driver.find_element('xpath',"//body").click()

    p = driver.find_element('xpath',"//div[@id='league-chemp']/table/thead/tr").text
    #print(p)
    playersHeader = driver.find_element('xpath',"//div[@id='league-players']/table/thead/tr").text


    i = 1
    l = []
    while True:
        try:
            b = driver.find_element('xpath',f"//div[@id='league-chemp']/table/tbody/tr[{i}]").text
            
            teams = [str for str in b.split(' ') if (str.isalpha())]
            #print(teams)
            #not "abc" in s
            
            c = get_numbers(b)
            c.insert(1, ' '.join(teams))
            
            l.append(c) 

            #print(l)
            #leaderboard.loc[i] = b.split(' ')
            i+=1
            
        except NoSuchElementException:
            break

    leaderboard = pd.DataFrame(l, columns=p.split(' '))
    leaderboard = leaderboard.set_index('№')
    #print('CLASSIFICA')
    #print(leaderboard)

    driver.find_element('name','players-date-start').click()


    sDate  = driver.find_element('name','players-date-start') # set start date input element into variable
    sDate.clear() #clear existing entry
    sDate.send_keys(s_date) #add custom entry


    driver.find_element('name','players-date-end').click()

    eDate  = driver.find_element('name','players-date-end') # set start date input element into variable
    eDate.clear() #clear existing entry
    eDate.send_keys(e_date) #add custom entry

    driver.find_element('id',"players-filter").click()


    time.sleep(2)

    pagination = driver.find_elements('xpath',"//ul//li[contains(@class,'page')]")

    #print('PAGINATION:')
    #print(pagination[-1].text)
    max_page = int(pagination[-1].text)-2
    j = int(pagination[3].text)
    


    #print('CLASSIFICA')
    #print(players_stats)


    l = []

    for i in range(0,4):
        if get_table(i,l,leaderboard,driver):
            break




    while True:
        #print('ciaoooooo')
        #print(j)
        
        if get_table(-2,l,leaderboard,driver) or (j == max_page):
            break
        j += 1



    get_table(-1,l,leaderboard,driver)

    #print(l)

    players_stats = pd.DataFrame(l, columns=playersHeader.split(' '))
    players_stats = players_stats.set_index('№')
    players_stats = players_stats.dropna()
    players_stats['Min'] = players_stats['Min'] * 1000
    leaderboard['W'] = leaderboard['W'] * 1000

    #input('Premi INVIO per chiudere l\'istanza di Chrome lanciata')
    driver.quit()

    

    return leaderboard,players_stats

<ipython-input-10-5eb279b5cd87>:20: DeprecationWarning: use options instead of chrome_options
  d = webdriver.Chrome(chrome,chrome_options=options)


In [102]:
l,p = get_tables('11/17/2020', '05/1/2021',d)

<ipython-input-101-c15c86caeb15>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary,chrome_options=options)
<ipython-input-101-c15c86caeb15>:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary,chrome_options=options)


In [103]:
l

,Team,M,W,D,L,G,GA,PTS,xG,xGA,xPTS
№,,,,,,,,,,,
1,Inter,27,22.0,4,1,58,18,70,54.21,25.39,56.95
2,Juventus,27,17.0,5,5,53,24,56,59.50,25.05,57.60
3,Atalanta,26,16.0,7,3,60,25,55,57.04,25.40,55.09
4,AC Milan,27,16.0,4,7,46,34,52,48.46,36.68,44.70
5,Napoli,27,16.0,3,8,58,33,51,52.02,33.70,48.31
6,Lazio,25,16.0,2,7,45,33,50,39.85,31.58,38.74
7,Roma,26,12.0,5,9,42,40,41,44.38,32.84,43.07
8,Sassuolo,26,10.0,7,9,37,41,37,38.66,39.16,36.18
9,Udinese,26,9.0,8,9,32,33,35,34.55,36.54,35.07


In [1]:
p

NameError: NameError: name 'p' is not defined

In [66]:
s1 = json.dumps(matches.json())
d2 = json.loads(s1)
#matches_df = pd.DataFrame(columns=['date','day','hteam_name','ateam_name','hteam_score','ateam_score'])
matches_df = pd.json_normalize(matches.json()['matches'])[['utcDate','matchday','homeTeam.shortName','awayTeam.shortName','score.fullTime.home','score.fullTime.away']]
matches_df = matches_df.rename(columns={'utcDate':'date','matchday':'day','homeTeam.shortName':'hteam_name','awayTeam.shortName':'ateam_name','score.fullTime.home':'hteam_score','score.fullTime.away':'ateam_score'})
matches_df = matches_df.sort_values(['day','date'])

In [79]:
matches_df[matches_df.day == 1]

,date,day,hteam_name,ateam_name,hteam_score,ateam_score
0,2020-09-19T16:00:00Z,1,Fiorentina,Torino,1,0
1,2020-09-19T18:45:00Z,1,Verona,Roma,3,0
2,2020-09-20T10:30:00Z,1,Parma,Napoli,0,2
3,2020-09-20T13:00:00Z,1,Genoa,Crotone,4,1
4,2020-09-20T16:00:00Z,1,Sassuolo,Cagliari,1,1
5,2020-09-20T18:45:00Z,1,Juventus,Sampdoria,3,0
6,2020-09-21T18:45:00Z,1,Milan,Bologna,2,0
17,2020-09-30T16:00:00Z,1,Udinese,Spezia Calcio,0,2
18,2020-09-30T16:00:00Z,1,Benevento,Inter,2,5
19,2020-09-30T18:45:00Z,1,Lazio,Atalanta,1,4


In [72]:
# importing libraries and packages
import snscrape.modules.twitter as sntwitter
import pandas as pd


# Creating list to append tweet data 
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('(Salernitana OR Empoli) -inter -milan -roma since:2022-08-28 until:2022-09-04 lang:it min_faves:0 -filter:links filter:verified').get_items()): #declare a username 
    if i>2000: #number of tweets you want to scrape
        break
    tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount]) #declare the attributes to be returned
    
# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes'])

In [73]:
df = tweets_df1.sort_values('Likes', ascending=False)
df

,Datetime,Tweet Id,Text,Username,Likes
32,2022-09-01 15:52:57+00:00,1565367115036729344,Piccola precisazione su #Pjaca: il croato è le...,romeoagresti,1376
38,2022-09-01 09:02:45+00:00,1565263883643805696,#Pjaca in prestito dalla #Juventus all'#Empoli...,GiovaAlbanese,1075
68,2022-08-31 13:03:53+00:00,1564962179144630273,"#Parisi, forte scatto della #Fiorentina. Dialo...",AlfredoPedulla,356
73,2022-08-30 18:06:35+00:00,1564675965086830595,"ULTIM'ORA #MERCATO\n#Empoli, in arrivo #Pjaca ...",SkySport,223
55,2022-08-31 17:03:28+00:00,1565022469278339073,In #EmpoliVerona primo gol in serie A per Tomm...,gippu1,207
...,...,...,...,...,...
54,2022-08-31 17:03:29+00:00,1565022475271946241,34' 0-0 #EmpoliHellasVerona \n🟨 Ammonito Cecch...,EmpoliCalcio,0
53,2022-08-31 17:05:09+00:00,1565022895952302080,35' 0-0 #EmpoliHellasVerona \nPunizione di Sto...,EmpoliCalcio,0
46,2022-08-31 17:55:26+00:00,1565035549773135873,67' 1-0 #EmpoliHellasVerona \nDoppio cambio Ve...,EmpoliCalcio,0
49,2022-08-31 17:47:44+00:00,1565033612155785222,59' 1-0 #EmpoliHellasVerona \nCambio di gioco ...,EmpoliCalcio,0


In [250]:

def create_string(hteam, ateam):

    all_teams = list(teams)

    other_teams = [f'({team})' for team in all_teams if (hteam not in team) and (ateam not in team)]

    query_str =  f'({hteam} {ateam})' + ' -' + ' -'.join(other_teams)
    
    return query_str


def create_date_range(date,range):
    
    date = date.split('T')[0]
    d = datetime.datetime.strptime(date, "%Y-%m-%d")
    delta = datetime.timedelta(days=-range)
    date_from = d + delta
    query_str = f' since:{date_from.date()} until:{date}'
    return query_str

#2020-09-21T18:45:00Z

#since:#2020-09-21T18:45:00Z  until:#2020-09-22T18:45:00Z

#since:2022-08-28 until:2022-09-04

def get_tweets(matchday):

    tweets = [None for x in range(10)]
    i = 0
    matches_day = matches_df[matches_df.day == matchday].values
    for match in matches_day:
        h_team = match[2]
        a_team = match[3]
        date = match[0]

        tweets[i] = get_match_tweets(h_team,a_team,date)
        
        
        i+=1

    return tweets

In [254]:

import datetime


def get_match_tweets(hteam,ateam,date):
    

    # Creating list to append tweet data 
    tweets_list1 = []

    print(create_string(hteam, ateam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links filter:verified')

    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(create_string(hteam, ateam) + create_date_range(date,21) + ' lang:it min_faves:0 -filter:links').get_items()): #declare a username 
        if i>1000: #number of tweets you want to scrape
            break
        tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount]) #declare the attributes to be returned
        
    # Creating a dataframe from the tweets list above 
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Likes'])

    start_time = datetime.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")
    
    tweets_df1 = tweets_df1[[( (datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").day == start_time.day) and (datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").hour < start_time.hour) or not(datetime.datetime.strptime(str(datex), "%Y-%m-%d %H:%M:%S+00:00").day == start_time.day)) for datex in tweets_df1.Datetime ]]

    print(tweets_df1.head(1))
    return tweets_df1.sort_values('Datetime')


In [255]:
#for i in range(38)
temp = get_tweets(1)

(Fiorentina Torino) -(Milan) -(Roma) -(Atalanta) -(Bologna) -(Cagliari) -(Genoa) -(Inter) -(Juventus) -(Lazio) -(Parma) -(Napoli) -(Udinese) -(Verona) -(Sassuolo) -(Crotone) -(Spezia Calcio) -(Sampdoria) -(Benevento) since:2020-08-29 until:2020-09-19 lang:it min_faves:0 -filter:links filter:verified
                   Datetime             Tweet Id  \
0 2020-09-18 16:00:47+00:00  1306986534353207296   

                                                Text        Username  Likes  
0  Mancano ventiquattro ore…e io ho già l’ansia a...  Loscoindividuo      2  
(Verona Roma) -(Milan) -(Fiorentina) -(Atalanta) -(Bologna) -(Cagliari) -(Genoa) -(Inter) -(Juventus) -(Lazio) -(Parma) -(Napoli) -(Udinese) -(Sassuolo) -(Crotone) -(Spezia Calcio) -(Sampdoria) -(Torino) -(Benevento) since:2020-08-29 until:2020-09-19 lang:it min_faves:0 -filter:links filter:verified
                   Datetime             Tweet Id  \
0 2020-09-18 23:57:01+00:00  1307106382110437378   

                                

KeyError: KeyError: 'Datetime'

In [252]:
temp[2]

NameError: NameError: name 'temp' is not defined

In [253]:
tweets_df1

,Datetime,Tweet Id,Text,Username,Likes


In [1]:
dataframe

NameError: NameError: name 'dataframe' is not defined

In [14]:
dataframe = pd.read_json(path_or_buf='./scrapedUrlsX.json')
dataframe

,date,tags,title,text
0,26/08/2020,"[serie a, roma, calciomercato, calciomercato r...","Pedro: 'Roma, voglio vincere ancora. Fonseca o...",Le prime parole dell'esterno spagnolo come n...
1,26/08/2020,"[fotogallery, juventus, maglie]","Juve, la terza maglia è ufficiale. FOTO | Sky ...",La Juventus ha ufficializzato la terza magli...
2,26/08/2020,"[fotogallery, calciomercato, calciomercato ser...","Serie A, i giocatori che hanno aumentato di pi...",Scorre il countdown per l'inizio del nuovo c...
3,26/08/2020,"[fotogallery, calciomercato, calciomercato ser...","Serie A, i giocatori che hanno perso più valor...",Un nuovo campionato alle porte e una session...
4,26/08/2020,"[napoli, serie a, ritiri serie a]",Osimhen show in ritiro: il rito di iniziazione...,Show dell'attaccante nigeriano che nel ritir...
...,...,...,...,...
4166,11/06/2021,"[juventus, serie a, aaron ramsey]",Ramsey: 'Le ultime due stagioni alla Juventus ...,Il centrocampista gallese in un’intervista h...
4167,25/06/2021,"[serie a, napoli, victor osimhen]","Osimhen, per il giocatore del Napoli murales e...",Ritorno da star per Victor Osimhen nella sua...
4168,25/06/2021,"[fotogallery, inter, serie a]","Inter, 40 tifosi vip hanno aderito al progetto...",Sono 40 i tifosi vip dell'Inter che hanno de...
4169,25/06/2021,"[fotogallery, juventus, serie a]","Juve, le anticipazioni della maglia da trasfer...",I siti specializzati 'Footy Headlines' e 'La...


In [7]:
ls 

chromedriver
environment.yml
google-chrome-stable_current_amd64.deb
google-chrome-stable_current_amd64.deb.1
google-chrome-stable_current_amd64.deb.2
google-chrome-stable_current_amd64.deb.3
lost+found/
skysport/
